In [192]:
import pandas as pd

**PreReq Detect Demo**

Here we will show off the basic idea behind the PreReq Detect application and how it would be used by a student at the U of I.
Let us first start with the the basic setup of the app.

**Data Ingestion**

At the current time, we do not have access to the data where these minor and certificate requirements are stored. In the mean time, we will create a sample of how the data may be stored in order to test out our concept

In [12]:
#Will need to adjust data setup once we get access to real data

requirements = pd.DataFrame({
                             'title' : ['minor_cs', 'certificate_ds'],
                             'required_courses' : [['cs_124', 'cs_128', 'cs_173', 'cs_225'], ['stat_107', 'stat_207']],
                             'choice_list_1' : [['cs_307', 'cs_340', 'cs_341', 'cs_357', 'cs_361', 'cs_374', 'cs_407', 'cs_409'], ['math_227', 'math_257']],
                             'choice_list_1_num' : [2, 1],
                             'choice_list_2' : [[], ['stat_385', 'stat_420', 'stat_440', 'stat_480']],
                             'choice_list_2_num' : [0, 1],
                             'choice_list_3' : [[], []],
                             'choice_list_3_num' : [0, 0],
                             'choice_list_4' : [[], []],
                             'choice_list_4_num' : [0, 0],
                             'choice_list_5' : [[], []],
                             'choice_list_5_num' : [0, 0]
})

In [13]:
display(requirements)

,title,required_courses,choice_list_1,choice_list_1_num,choice_list_2,choice_list_2_num,choice_list_3,choice_list_3_num,choice_list_4,choice_list_4_num,choice_list_5,choice_list_5_num
0,minor_cs,"[cs_124, cs_128, cs_173, cs_225]","[cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2,[],0,[],0,[],0,[],0
1,certificate_ds,"[stat_107, stat_207]","[math_227, math_257]",1,"[stat_385, stat_420, stat_440, stat_480]",1,[],0,[],0,[],0


We now will generate some potential students and see if we can use them to determine if they have any of the requirements needed to qualify at all for the listed minors/certificates.

In [14]:
#Completed required courses
student_1 = ['cs_124', 'cs_128', 'cs_173', 'cs_225']

#Completed enough elective courses
student_2 = ['cs_307', 'cs_340', 'cs_341']

#Completed some required and some elective courses
student_3 = ['cs_124', 'cs_307']

#Completed neither any required or elective courses
student_4 = ['stat_107']

We create this function to be used as a checker for both the required and elective course lists. 

In [157]:
def req_finder(requirements, student):
    cols = ['choice_list_1','choice_list_2','choice_list_3','choice_list_4','choice_list_5']
    
    filtered_requirements = requirements[requirements['required_courses'].apply(lambda x: any(item in student for item in x))]
    filtered_requirements = filtered_requirements.copy()
    filtered_requirements['required_courses'] = filtered_requirements['required_courses'].apply(tuple)

    overall_list = filtered_requirements.copy()

    for col_name in cols:
        overall_list = req_finder_helper(requirements, overall_list, col_name, student)
        overall_list = overall_list.copy()
        overall_list[col_name] = overall_list[col_name].apply(tuple)
        
    overall_tuples = overall_list.apply(lambda col: col.map(lambda x: tuple(x) if isinstance(x, list) else x))
    overall_no_dupes = overall_tuples.drop_duplicates()

    return overall_no_dupes

We create this function to help simplify the function of the req_finder.

In [158]:
def req_finder_helper(requirements, data, list_name, student):
    filtered_choice_list = requirements[requirements[list_name].apply(lambda x: any(item in student for item in x))]
    filtered_choice_list = filtered_choice_list.copy()
    filtered_choice_list[list_name] = filtered_choice_list[list_name].apply(tuple)
    filtered_courses = pd.concat([filtered_choice_list, data])
    return filtered_courses
    

In [159]:
test1 = req_finder(requirements, student_1)
display(test1)

,title,required_courses,choice_list_1,choice_list_1_num,choice_list_2,choice_list_2_num,choice_list_3,choice_list_3_num,choice_list_4,choice_list_4_num,choice_list_5,choice_list_5_num
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2,(),0,(),0,(),0,(),0


In [160]:
test2 = req_finder(requirements, student_2)
display(test2)

,title,required_courses,choice_list_1,choice_list_1_num,choice_list_2,choice_list_2_num,choice_list_3,choice_list_3_num,choice_list_4,choice_list_4_num,choice_list_5,choice_list_5_num
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2,(),0,(),0,(),0,(),0


In [161]:
test3 = req_finder(requirements, student_3)
display(test3)

,title,required_courses,choice_list_1,choice_list_1_num,choice_list_2,choice_list_2_num,choice_list_3,choice_list_3_num,choice_list_4,choice_list_4_num,choice_list_5,choice_list_5_num
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2,(),0,(),0,(),0,(),0


In [162]:
test4 = req_finder(requirements, student_4)
display(test4)

,title,required_courses,choice_list_1,choice_list_1_num,choice_list_2,choice_list_2_num,choice_list_3,choice_list_3_num,choice_list_4,choice_list_4_num,choice_list_5,choice_list_5_num
1,certificate_ds,"(stat_107, stat_207)","(math_227, math_257)",1,"(stat_385, stat_420, stat_440, stat_480)",1,(),0,(),0,(),0


Using the req_finder, we now can see the minors/certificates relevant to a student's currently completed coursework. We now will try and create separation between the courses they have completed and what remains for required courses and elective courses. We will use student_3 to test this since they have completed a limited amount in both the required course and elective course columns.

In [163]:
student_3_courses = req_finder(requirements, student_3)
display(student_3_courses)

,title,required_courses,choice_list_1,choice_list_1_num,choice_list_2,choice_list_2_num,choice_list_3,choice_list_3_num,choice_list_4,choice_list_4_num,choice_list_5,choice_list_5_num
0,minor_cs,"(cs_124, cs_128, cs_173, cs_225)","(cs_307, cs_340, cs_341, cs_357, cs_361, cs_37...",2,(),0,(),0,(),0,(),0


We create this function to separate between classes completed toward the given minor/certificate from those that still would need to be completed.

In [164]:
def completion_separator(student_courses, student):
    cols = ['choice_list_1','choice_list_2','choice_list_3','choice_list_4','choice_list_5']
    
    student_courses['completed_required_courses'] = student_courses['required_courses'].apply(lambda x: tuple(value for value in x if value in student))
    student_courses['remaining_required_courses'] = student_courses['required_courses'].apply(lambda x: tuple(value for value in x if value not in student))

    for col_name in cols:
        student_courses = completion_separator_choice_helper(student_courses,col_name, student)

    student_courses = student_courses.drop(columns={'required_courses', 
                                                    'choice_list_1', 
                                                    'choice_list_1_num', 
                                                    'choice_list_2', 
                                                    'choice_list_2_num',
                                                    'choice_list_3', 
                                                    'choice_list_3_num',
                                                    'choice_list_4', 
                                                    'choice_list_4_num',
                                                    'choice_list_5', 
                                                    'choice_list_5_num'
                                                })

    return student_courses

We create this function to help the completion separator perform its separation for each of the choice lists.

In [238]:
def completion_separator_choice_helper(data, list_name, student):
    data['completed_'+list_name] = data[list_name].apply(lambda x: tuple(value for value in x if value in student))
    data['remaining_'+list_name] = data[list_name].apply(lambda x: tuple(value for value in x if value not in student))

    data[list_name+'_num_completed'] = data['completed_'+list_name].apply(lambda x: 0 if x == () else len(x))
    data[list_name+'_num_remaining'] = data[list_name+'_num'] - data[list_name+'_num_completed']
    data[list_name+'_num_remaining'] = data[list_name+'_num_remaining'].apply(lambda x: max(x, 0))
    
    return data

In [236]:
separation_3 = completion_separator(student_3_courses, student_3)
display(separation_3)

,title,completed_required_courses,remaining_required_courses,completed_choice_list_1,remaining_choice_list_1,choice_list_1_num_completed,choice_list_1_num_remaining,completed_choice_list_2,remaining_choice_list_2,choice_list_2_num_completed,...,choice_list_3_num_completed,choice_list_3_num_remaining,completed_choice_list_4,remaining_choice_list_4,choice_list_4_num_completed,choice_list_4_num_remaining,completed_choice_list_5,remaining_choice_list_5,choice_list_5_num_completed,choice_list_5_num_remaining
0,minor_cs,"(cs_124,)","(cs_128, cs_173, cs_225)","(cs_307,)","(cs_340, cs_341, cs_357, cs_361, cs_374, cs_40...",1,1,(),(),0,...,0,0,(),(),0,0,(),(),0,0


Let's see this for a new student, one that has requirements for both **minor_cs** and **certificate_ds**.

In [237]:
student_5 = ['stat_107', 'math_257', 'stat_440', 'cs_128', 'cs_307', 'stat_207', 'cs_225']
student_5_courses = req_finder(requirements, student_5)
separation_5 = completion_separator(student_5_courses, student_5)
display(separation_5)

,title,completed_required_courses,remaining_required_courses,completed_choice_list_1,remaining_choice_list_1,choice_list_1_num_completed,choice_list_1_num_remaining,completed_choice_list_2,remaining_choice_list_2,choice_list_2_num_completed,...,choice_list_3_num_completed,choice_list_3_num_remaining,completed_choice_list_4,remaining_choice_list_4,choice_list_4_num_completed,choice_list_4_num_remaining,completed_choice_list_5,remaining_choice_list_5,choice_list_5_num_completed,choice_list_5_num_remaining
1,certificate_ds,"(stat_107, stat_207)",(),"(math_257,)","(math_227,)",1,0,"(stat_440,)","(stat_385, stat_420, stat_480)",1,...,0,0,(),(),0,0,(),(),0,0
0,minor_cs,"(cs_128, cs_225)","(cs_124, cs_173)","(cs_307,)","(cs_340, cs_341, cs_357, cs_361, cs_374, cs_40...",1,1,(),(),0,...,0,0,(),(),0,0,(),(),0,0
